In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=aea2a1923b5b0deb42ef9967114cbdc4c17058bf864f4af0b7ed4a4eef14dd5b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('sqlspark').getOrCreate()

In [ ]:
bestseller=spark.read.json('/content/nyt2.json')

In [ ]:
bestseller.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|     

In [ ]:
bestseller.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- amazon_product_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- bestsellers_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- published_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rank: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- rank_last_week: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- title: string (nullable = true)
 |-- weeks_on_list: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)



In [ ]:
bestseller.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [ ]:
# To remove Dupliactes
bestsellerdf=bestseller.dropDuplicates()

In [ ]:
bestsellerdf.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [ ]:
# Querying operations such as subsetting columns with "Select", adding conditions with "When" and 
# filtering column contents with "Like"

In [ ]:
# Select only "author" variable
bestsellerdf.select('author').show(10)

+------------------+
|            author|
+------------------+
|      Daniel Silva|
|        Jane Green|
|          J D Robb|
|     Toni Morrison|
|   Nicholas Sparks|
|   Stephenie Meyer|
|          Lisa See|
|    David Baldacci|
|Laurell K Hamilton|
|     Leila Meacham|
+------------------+
only showing top 10 rows



In [ ]:
bestsellerdf.select("author").distinct().count()   # Unique authors

738

In [ ]:
bestsellerdf.select("author","title","rank","price").show(10)

+------------------+-------------------+----+-------------+
|            author|              title|rank|        price|
+------------------+-------------------+----+-------------+
|      Daniel Silva| THE SECRET SERVANT| {1}|{26.95, null}|
|        Jane Green|    THE BEACH HOUSE|{18}|    {null, 0}|
|          J D Robb| SALVATION IN DEATH| {2}|{25.95, null}|
|     Toni Morrison|            A MERCY| {5}|{23.95, null}|
|   Nicholas Sparks|      THE LUCKY ONE|{16}|    {null, 0}|
|   Stephenie Meyer|           THE HOST| {5}|{25.99, null}|
|          Lisa See|     SHANGHAI GIRLS| {4}|   {null, 25}|
|    David Baldacci|          TRUE BLUE|{14}|{27.99, null}|
|Laurell K Hamilton|DIVINE MISDEMEANORS|{18}|    {null, 0}|
|     Leila Meacham|              ROSES| {9}|{24.99, null}|
+------------------+-------------------+----+-------------+
only showing top 10 rows



In [ ]:
bestsellerdf.groupBy("author").count().show()

+--------------------+-----+
|              author|count|
+--------------------+-----+
|    Elin Hilderbrand|   58|
|         Lisa Genova|    7|
|   Sharon Kay Penman|    2|
|          James Frey|    2|
|        Will Allison|    1|
|          Tea Obreht|    6|
|   Patricia Cornwell|   64|
|       Laurie R King|    6|
|        Sarah Dunant|    1|
|        Tim Johnston|    1|
|          Sara Gruen|   13|
|Tom Clancy with P...|    9|
|        Terry Brooks|   19|
|        Lisa Wingate|   42|
|        Daniel Silva|   69|
|Karen White, Beat...|    1|
|      Rachel Kushner|    3|
|      Jackie Collins|   14|
|          Pat Conroy|   11|
|          Ann B Ross|    5|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Which author has most books published?
bestsellerdf.groupBy("author").count().sort("count",ascending=False).show()

+--------------------+-----+
|              author|count|
+--------------------+-----+
|        John Grisham|  226|
|      David Baldacci|  222|
|      Danielle Steel|  174|
|        Stephen King|  157|
|       Anthony Doerr|  141|
|     Nicholas Sparks|  136|
|      Kristin Hannah|  128|
|       Stieg Larsson|  121|
|James Patterson a...|  119|
|   George R R Martin|  119|
|     James Patterson|  115|
|    Kathryn Stockett|  114|
|     Janet Evanovich|  111|
|       John Sandford|  107|
|        Stuart Woods|  107|
|    Michael Connelly|  104|
|       Paula Hawkins|   97|
|           Lee Child|   94|
|    Charlaine Harris|   94|
|        Jodi Picoult|   90|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Print all titles published by John Grisham
bestsellerdf.select("author","title","rank").filter(bestsellerdf.author=="John Grisham").show()

+------------+--------------+----+
|      author|         title|rank|
+------------+--------------+----+
|John Grisham| THE RACKETEER| {5}|
|John Grisham| GRAY MOUNTAIN| {3}|
|John Grisham|  ROGUE LAWYER|{10}|
|John Grisham| THE ASSOCIATE| {2}|
|John Grisham|THE CONFESSION| {9}|
|John Grisham|THE LITIGATORS| {8}|
|John Grisham| GRAY MOUNTAIN| {2}|
|John Grisham|   FORD COUNTY| {7}|
|John Grisham|  SYCAMORE ROW| {5}|
|John Grisham| GRAY MOUNTAIN| {2}|
|John Grisham|  ROGUE LAWYER| {1}|
|John Grisham|  ROGUE LAWYER| {3}|
|John Grisham|   FORD COUNTY| {4}|
|John Grisham|    CALICO JOE| {9}|
|John Grisham|THE CONFESSION| {2}|
|John Grisham| GRAY MOUNTAIN| {1}|
|John Grisham| THE RACKETEER| {3}|
|John Grisham| THE RACKETEER| {3}|
|John Grisham|  SYCAMORE ROW| {5}|
|John Grisham| THE ASSOCIATE| {3}|
+------------+--------------+----+
only showing top 20 rows



In [ ]:
# Print all titles published by John Grisham
bestsellerdf.select("author","title","rank").where(bestsellerdf.author=="John Grisham").dropDuplicates(['title']).sort("rank").show()
# Where & filter are same or alias

+------------+---------------+----+
|      author|          title|rank|
+------------+---------------+----+
|John Grisham|  CAMINO ISLAND| {1}|
|John Grisham|   ROGUE LAWYER|{10}|
|John Grisham|     THE APPEAL|{19}|
|John Grisham|  THE ASSOCIATE| {2}|
|John Grisham|THE ROOSTER BAR| {2}|
|John Grisham|  GRAY MOUNTAIN| {3}|
|John Grisham|  THE RACKETEER| {5}|
|John Grisham|   SYCAMORE ROW| {5}|
|John Grisham|    FORD COUNTY| {7}|
|John Grisham|   THE WHISTLER| {7}|
|John Grisham| THE LITIGATORS| {8}|
|John Grisham|     CALICO JOE| {9}|
|John Grisham| THE CONFESSION| {9}|
+------------+---------------+----+



In [ ]:
# Print all titles published by John Grisham distinct books titles
bestsellerdf.select("author","title","rank").where(bestsellerdf.author=="John Grisham").dropDuplicates(['title']).distinct().show()

+------------+---------------+----+
|      author|          title|rank|
+------------+---------------+----+
|John Grisham|     CALICO JOE| {9}|
|John Grisham|  CAMINO ISLAND| {1}|
|John Grisham|    FORD COUNTY| {7}|
|John Grisham|  GRAY MOUNTAIN| {3}|
|John Grisham|   ROGUE LAWYER|{10}|
|John Grisham|   SYCAMORE ROW| {5}|
|John Grisham|     THE APPEAL|{19}|
|John Grisham|  THE ASSOCIATE| {2}|
|John Grisham| THE CONFESSION| {9}|
|John Grisham| THE LITIGATORS| {8}|
|John Grisham|  THE RACKETEER| {5}|
|John Grisham|THE ROOSTER BAR| {2}|
|John Grisham|   THE WHISTLER| {7}|
+------------+---------------+----+



In [ ]:
# Print all titles published by John Grisham unique books
bestsellerdf.select("author","title","rank").where(bestsellerdf.author=="John Grisham").dropDuplicates(['title']).distinct().count()

13

In [ ]:
# Distinct titles publishing by "David Baldacci"
bestsellerdf.select("author","title","rank").where(bestsellerdf.author=="David Baldacci").dropDuplicates(['title']).distinct().show(25)

+--------------+--------------------+----+
|        author|               title|rank|
+--------------+--------------------+----+
|David Baldacci|DELIVER US FROM EVIL|{17}|
|David Baldacci|      DIVINE JUSTICE|{11}|
|David Baldacci|            END GAME|{14}|
|David Baldacci|        FIRST FAMILY| {8}|
|David Baldacci|       HELL'S CORNER|{13}|
|David Baldacci|    KING AND MAXWELL| {5}|
|David Baldacci|          MEMORY MAN|{20}|
|David Baldacci|       NO MAN'S LAND| {6}|
|David Baldacci|          ONE SUMMER| {5}|
|David Baldacci|          THE ESCAPE|{11}|
|David Baldacci|          THE FALLEN| {2}|
|David Baldacci|             THE FIX| {1}|
|David Baldacci|       THE FORGOTTEN|{11}|
|David Baldacci|          THE GUILTY| {6}|
|David Baldacci|             THE HIT| {2}|
|David Baldacci|        THE INNOCENT| {8}|
|David Baldacci|       THE LAST MILE| {5}|
|David Baldacci|       THE SIXTH MAN| {3}|
|David Baldacci|          THE TARGET|{13}|
|David Baldacci|     THE WHOLE TRUTH|{10}|
|David Bald

In [ ]:
# Print all titles published by "David Baldacci" unique books
bestsellerdf.select("author","title","rank").where(bestsellerdf.author=="David Baldacci").dropDuplicates(['title']).distinct().count()

22

In [ ]:
# Books published by Stephen King and Danielle Steel
bestsellerdf.select("author","title","rank").where(bestsellerdf.author.isin("Stephen King","Danielle Steel")).dropDuplicates(['title']).show()

+--------------+-----------------+----+
|        author|            title|rank|
+--------------+-----------------+----+
|  Stephen King|         11/22/63| {6}|
|Danielle Steel|44 CHARLES STREET|{11}|
|Danielle Steel|     A GOOD WOMAN|{18}|
|Danielle Steel|   A PERFECT LIFE| {4}|
|Danielle Steel|ACCIDENTAL HEROES| {1}|
|Danielle Steel| AGAINST ALL ODDS| {3}|
|Danielle Steel|         BETRAYAL| {7}|
|Danielle Steel|         BIG GIRL| {5}|
|  Stephen King|   BLOCKADE BILLY|{13}|
|Danielle Steel|             BLUE| {2}|
|Danielle Steel|          COUNTRY|{12}|
|Danielle Steel|  DANGEROUS GAMES| {7}|
|  Stephen King|     DOCTOR SLEEP| {2}|
|  Stephen King|     END OF WATCH|{18}|
|Danielle Steel|        FAIRYTALE|{12}|
|Danielle Steel|  FALL FROM GRACE| {2}|
|Danielle Steel|      FAMILY TIES|{20}|
|  Stephen King|  FINDERS KEEPERS|{13}|
|Danielle Steel|      FIRST SIGHT| {8}|
|Danielle Steel|  FRIENDS FOREVER|{11}|
+--------------+-----------------+----+
only showing top 20 rows



In [ ]:
# Books published by Stephen King and Danielle Steel
bestsellerdf.select("author","title","rank").where(bestsellerdf.author.isin(["Stephen King","Danielle Steel"])).dropDuplicates(['title']).sort('author').show(100)

+--------------+--------------------+----+
|        author|               title|rank|
+--------------+--------------------+----+
|Danielle Steel|   ACCIDENTAL HEROES| {1}|
|Danielle Steel|            BIG GIRL| {5}|
|Danielle Steel|                BLUE| {2}|
|Danielle Steel|             COUNTRY|{12}|
|Danielle Steel|   44 CHARLES STREET|{11}|
|Danielle Steel|         FIRST SIGHT| {8}|
|Danielle Steel|     FRIENDS FOREVER|{11}|
|Danielle Steel|      HAPPY BIRTHDAY|{16}|
|Danielle Steel|       HOTEL VENDOME|{18}|
|Danielle Steel|              LEGACY|{12}|
|Danielle Steel|               MAGIC| {5}|
|Danielle Steel|     DANGEROUS GAMES| {7}|
|Danielle Steel|        A GOOD WOMAN|{18}|
|Danielle Steel|            BETRAYAL| {7}|
|Danielle Steel|PROPERTY OF A NOB...| {4}|
|Danielle Steel|               ROGUE| {4}|
|Danielle Steel|      RUSHING WATERS| {6}|
|Danielle Steel|     SOUTHERN LIGHTS| {8}|
|Danielle Steel|       THE APARTMENT| {3}|
|Danielle Steel|           THE AWARD| {4}|
|Danielle S

In [ ]:
# Count of books that start wuth word "THE"
bestsellerdf.select("author","title","rank").where(bestsellerdf.title.contains("THE")).show()

+--------------------+--------------------+----+
|              author|               title|rank|
+--------------------+--------------------+----+
|        Daniel Silva|  THE SECRET SERVANT| {1}|
|          Jane Green|     THE BEACH HOUSE|{18}|
|     Nicholas Sparks|       THE LUCKY ONE|{16}|
|     Stephenie Meyer|            THE HOST| {5}|
|        Robert Crais|      THE FIRST RULE|{19}|
|        Aimee Bender|THE PARTICULAR SA...|{15}|
|        Robert Crais|          THE SENTRY| {8}|
|          Tea Obreht|    THE TIGER’S WIFE|{20}|
|        Tom Perrotta|       THE LEFTOVERS|{18}|
|         Umberto Eco| THE PRAGUE CEMETERY|{18}|
|       Julian Barnes|THE SENSE OF AN E...|{18}|
|  Karen Marie Moning|   INTO THE DREAMING|{14}|
|Robert Kirkman an...|THE WALKING DEAD,...|{11}|
|        John Grisham|       THE RACKETEER| {5}|
|            C  J Box|         THE HIGHWAY| {7}|
|    Robert Galbraith|THE CUCKOO'S CALLING| {3}|
|        Louise Penny|HOW THE LIGHT GET...|{20}|
|     Nicholas Spark

In [ ]:
# Count of books that start wuth word "THE"
bestsellerdf.select("author","title","rank").where(bestsellerdf.title.contains("THE")).distinct().count()

2940

In [ ]:
bestsellerdf.select("author","title","rank").where(bestsellerdf.title.contains("THE")).distinct().show(50)

+--------------------+--------------------+----+
|              author|               title|rank|
+--------------------+--------------------+----+
|           Brad Thor|    THE LAST PATRIOT|{10}|
|        S M Stirling|  THE SCOURGE OF GOD|{19}|
|       Sean Williams| THE FORCE UNLEASHED|{14}|
|        Lorrie Moore|A GATE AT THE STAIRS|{10}|
|     Nicholas Sparks|       THE LAST SONG|{10}|
|        Stephen King|UNDER THE DOME, P...| {4}|
|         Jean M Auel|THE LAND OF PAINT...| {8}|
|          Anne Tyler|THE BEGINNER’S GO...|{16}|
|         Vince Flynn|        THE LAST MAN| {4}|
|        Jodi Picoult|     THE STORYTELLER|{14}|
|     Nicholas Sparks|    THE LONGEST RIDE| {7}|
|        Ian Caldwell|    THE FIFTH GOSPEL|{12}|
|      Salman Rushdie|THE ENCHANTRESS O...|{20}|
|    Kathryn Stockett|            THE HELP| {5}|
|   Patricia Cornwell|THE SCARPETTA FACTOR|{12}|
|     James Lee Burke|   THE GLASS RAINBOW|{15}|
|          Glenn Beck|  THE OVERTON WINDOW|{20}|
|James Patterson a..

In [ ]:
bestsellerdf.select("author","title","rank").where(bestsellerdf.title.contains("THE")).dropDuplicates(['title']).distinct().count()

800

In [ ]:
bestsellerdf.select("author","title","rank").where(bestsellerdf.title.startswith("THE")).dropDuplicates(['title']).show(50)

+--------------------+--------------------+----+
|              author|               title|rank|
+--------------------+--------------------+----+
|         Steve Berry|     THE 14TH COLONY|{12}|
|James Patterson a...|    THE 17TH SUSPECT|{14}|
|     David Ebershoff|       THE 19TH WIFE|{17}|
|       James Rollins|  THE 6TH EXTINCTION|{20}|
|James Patterson a...|  THE 8TH CONFESSION|{16}|
|           Tami Hoag|        THE 9TH GIRL|{20}|
|James Patterson a...|    THE 9TH JUDGMENT| {1}|
|        Chris Pavone|        THE ACCIDENT|{15}|
|      Allison Pataki|THE ACCIDENTAL EM...|{12}|
|   Joyce Carol Oates|        THE ACCURSED|{13}|
|         Jim Butcher|THE AERONAUT'S WI...|{16}|
|           Lee Child|          THE AFFAIR| {2}|
|Karen Thompson Wa...| THE AGE OF MIRACLES|{10}|
|        Fannie Flagg|THE ALL-GIRL FILL...|{16}|
|   Brandon Sanderson|    THE ALLOY OF LAW| {7}|
| Jennifer Chiaverini|     THE ALOHA QUILT|{11}|
|        Jim Harrison|THE ANCIENT MINSTREL|{14}|
|   Carlos Ruiz Zafó

In [ ]:
from pyspark.sql.functions import when,lit

In [ ]:
# Create a New Column called Theme with love representing romance
bestsellerdf=bestsellerdf.withColumn("Theme",when(bestsellerdf.title.contains("LOVE"),"Romance").otherwise("NonRomance"))

In [ ]:
bestsellerdf[bestsellerdf.Theme=='Romance'].count()

105

In [ ]:
# Create a New Column called horror_theme with words like "horror" or "mystery" - horror or nonhorror
bestsellerdf.select("title").show(100,truncate=False)

+-------------------------------------------------+
|title                                            |
+-------------------------------------------------+
|THE SECRET SERVANT                               |
|THE BEACH HOUSE                                  |
|SALVATION IN DEATH                               |
|A MERCY                                          |
|THE LUCKY ONE                                    |
|THE HOST                                         |
|SHANGHAI GIRLS                                   |
|TRUE BLUE                                        |
|DIVINE MISDEMEANORS                              |
|ROSES                                            |
|THE FIRST RULE                                   |
|ANGELOLOGY                                       |
|ABRAHAM LINCOLN: VAMPIRE HUNTER                  |
|THE PARTICULAR SADNESS OF LEMON CAKE             |
|THE SENTRY                                       |
|THE TIGER’S WIFE                                 |
|THE LEFTOVE

In [ ]:
bestsellerdf.select("author","title","rank").where(
    bestsellerdf.title.startswith("THE")).dropDuplicates(['title']).show(50)

+--------------------+--------------------+----+
|              author|               title|rank|
+--------------------+--------------------+----+
|         Steve Berry|     THE 14TH COLONY|{12}|
|James Patterson a...|    THE 17TH SUSPECT|{14}|
|     David Ebershoff|       THE 19TH WIFE|{17}|
|       James Rollins|  THE 6TH EXTINCTION|{20}|
|James Patterson a...|  THE 8TH CONFESSION|{16}|
|           Tami Hoag|        THE 9TH GIRL|{20}|
|James Patterson a...|    THE 9TH JUDGMENT| {1}|
|        Chris Pavone|        THE ACCIDENT|{15}|
|      Allison Pataki|THE ACCIDENTAL EM...|{12}|
|   Joyce Carol Oates|        THE ACCURSED|{13}|
|         Jim Butcher|THE AERONAUT'S WI...|{16}|
|           Lee Child|          THE AFFAIR| {2}|
|Karen Thompson Wa...| THE AGE OF MIRACLES|{10}|
|        Fannie Flagg|THE ALL-GIRL FILL...|{16}|
|   Brandon Sanderson|    THE ALLOY OF LAW| {7}|
| Jennifer Chiaverini|     THE ALOHA QUILT|{11}|
|        Jim Harrison|THE ANCIENT MINSTREL|{14}|
|   Carlos Ruiz Zafó

In [ ]:
# Create a New Column called Theme with love representing romance
from pyspark.sql.functions import when,lit
bestsellerdf=bestsellerdf.withColumn("Theme", when(bestsellerdf.description.contains("love")|
                                                   bestsellerdf.description.contains("romance"),"Romance").otherwise("NonRomance"))

In [ ]:
bestsellerdf[bestsellerdf.Theme=='Romance'].count()

443

In [ ]:
bestsellerdf.select("title").where(bestsellerdf.title.contains("FAMILY")).show(100,truncate=False)

+---------------------------+
|title                      |
+---------------------------+
|FAMILY PICTURES            |
|FIRST FAMILY               |
|AN O'BRIEN FAMILY CHRISTMAS|
|FIRST FAMILY               |
|DEAD IN THE FAMILY         |
|FAMILY TREE                |
|DEAD IN THE FAMILY         |
|DEAD IN THE FAMILY         |
|DEAD IN THE FAMILY         |
|FIRST FAMILY               |
|DEAD IN THE FAMILY         |
|DEAD IN THE FAMILY         |
|DEAD IN THE FAMILY         |
|FAMILY JEWELS              |
|DEAD IN THE FAMILY         |
|FAMILY JEWELS              |
|FIRST FAMILY               |
|THE FAMILY CORLEONE        |
|DID YOU EVER HAVE A FAMILY |
|FAMILY TIES                |
|FIRST FAMILY               |
|DEAD IN THE FAMILY         |
|THE FAMILY CORLEONE        |
|FIRST FAMILY               |
|DEAD IN THE FAMILY         |
|THE FAMILY CORLEONE        |
|DEAD IN THE FAMILY         |
|FAMILY TIES                |
|DEAD IN THE FAMILY         |
|DEAD IN THE FAMILY         |
|FIRST FAM

In [ ]:
bestsellerdf.select('description').show(30,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                       |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Gabriel Allon, an art restorer and an occasional spy for the Israeli secret service, uncovers a Russian arms sales plot.                                                          |
|A woman’s life changes when she rents out rooms in her Nantucket house.                                                                                                           |
|Lt. Eve Dallas investigates the murder of a priest who was not who he seemed; by Nora Roberts,